# 推薦系統的模型驗證

## 排序指標

### MRR@k

In [8]:
from typing import List

import numpy as np
import pandas as pd

In [9]:
def rr_at_k(one_user_relevances: List[int], k: int) -> float:
    """ Reciprocal Rank at K """
    for rank, relevance in enumerate(one_user_relevances[:k], start=1):
        # 正樣本
        if relevance == 1:
            return 1.0 / rank
    # 未命中正樣本
    return 0.0


def mrr_at_k(all_user_relevances: List[List[int]], k: int) -> float:
    """ Mean Reciprocal Rank at K """
    return np.mean([rr_at_k(one_user_relevances, k) for one_user_relevances in all_user_relevances])

In [10]:
## 測試樣本

one_user_relevances = [0, 0, 1, 0, 1]
k = 5
print(f"RR@{k}: {rr_at_k(one_user_relevances, k)}")

all_user_relevances = [
    [0, 0, 1, 0, 1],
    [1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0],
]
for user_relevances in all_user_relevances:
    print(f"User relevances: {user_relevances} -> RR@{k}: {rr_at_k(user_relevances, k)}")
print(f"MRR@{k}: {mrr_at_k(all_user_relevances=all_user_relevances, k=k)}")

RR@5: 0.3333333333333333
User relevances: [0, 0, 1, 0, 1] -> RR@5: 0.3333333333333333
User relevances: [1, 0, 0, 0, 0] -> RR@5: 1.0
User relevances: [0, 0, 0, 1, 0] -> RR@5: 0.25
MRR@5: 0.5277777777777778


### MAP@k(Mean Average Precision)

In [11]:
def ap_at_k(user_relevances: List[int], k: int) -> float:
    """  Average Precision at K """
    hits = 0
    ans = 0
    for rank, relevance in enumerate(user_relevances[:k], start=1):
        if relevance == 1:
            hits += 1
            ans += hits / rank

    return ans / hits if hits > 0 else 0.0

def map_at_k(all_user_relevances: List[List[int]], k: int) -> float:
    """ Mean Average Precision at K """
    return np.mean([ap_at_k(user_relevances, k) for user_relevances in all_user_relevances])

In [12]:
## 測試樣本

one_user_relevances = [0, 0, 1, 0, 1]
k = 5
print(f"AP@{k}: {ap_at_k(one_user_relevances, k)}")

all_user_relevances = [
    [0, 0, 1, 0, 1],
    [1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0],
]
for user_relevances in all_user_relevances:
    print(f"User relevances: {user_relevances} -> AP@{k}: {ap_at_k(user_relevances, k)}")
print(f"MAP@{k}: {map_at_k(all_user_relevances=all_user_relevances, k=k)}")

AP@5: 0.3666666666666667
User relevances: [0, 0, 1, 0, 1] -> AP@5: 0.3666666666666667
User relevances: [1, 0, 0, 0, 0] -> AP@5: 1.0
User relevances: [0, 0, 0, 1, 0] -> AP@5: 0.25
MAP@5: 0.5388888888888889


### NDCG

In [13]:
def dcg_at_k(user_relevances: List[int], k: int) -> float:
    """ Discounted Cumulative Gain at K """
    dcg = 0.0
    for rank, relevance in enumerate(user_relevances[:k], start=1):
        if relevance != 0:
            dcg += (relevance) / np.log2(rank+1)
    return dcg

def ndcg_at_k(all_user_relevances: List[List[int]], k: int) -> float:
    """ Normalized Discounted Cumulative Gain at K """
    return np.mean([
        dcg_at_k(user_relevances, k) / dcg_at_k(list(sorted(user_relevances[:k], reverse=True)), k)
    for user_relevances in all_user_relevances ] )

In [14]:
## 測試樣本

one_user_relevances = [0, 0, 1, 0, 1]
k = 5
print(f"DCG@{k}: {dcg_at_k(one_user_relevances, k)}")

all_user_relevances = [
    [0, 0, 1, 0, 1],
    [1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0],
]
for user_relevances in all_user_relevances:
    dcg = dcg_at_k(user_relevances, k)
    idcg = dcg_at_k(list(sorted(user_relevances[:k], reverse=True)), k)
    print(f"User relevances: {user_relevances} -> DCG@{k}: {dcg}")
    print(f"User relevances: {user_relevances} -> IDCG@{k}: {idcg}")
    print(f"NDCG: {dcg/idcg if idcg > 0 else 0.0}")
    print()
print(f"NDCG@{k}: {ndcg_at_k(all_user_relevances=all_user_relevances, k=k)}")

DCG@5: 0.8868528072345416
User relevances: [0, 0, 1, 0, 1] -> DCG@5: 0.8868528072345416
User relevances: [0, 0, 1, 0, 1] -> IDCG@5: 1.6309297535714575
NDCG: 0.5437713091520254

User relevances: [1, 0, 0, 0, 0] -> DCG@5: 1.0
User relevances: [1, 0, 0, 0, 0] -> IDCG@5: 1.0
NDCG: 1.0

User relevances: [0, 0, 0, 1, 0] -> DCG@5: 0.43067655807339306
User relevances: [0, 0, 0, 1, 0] -> IDCG@5: 1.0
NDCG: 0.43067655807339306

NDCG@5: 0.6581492890751395


## 其他評價指標
- 除了排序之外，推薦系統除了準之外，有時候為了破除 filter bubble 會在乎新奇性、多樣性，因為對於長期用戶在此APP/產品中會有幫助。

### 新奇性(Novelty)

In [15]:
def novelty_at_k(probabilities_of_recommendated_items: List[float], k: int) -> float:
    """ Novelty at K """
    num_of_items = min(k, len(probabilities_of_recommendated_items))
    if num_of_items == 0:
        return 0.0
    novelty = 0.0
    for p in probabilities_of_recommendated_items[:k]:
        if p == 0:
            novelty += 0.0
        else:
            novelty += -np.log2(p)
    return novelty / num_of_items

In [16]:
## 測試樣本

low_probs = [0.001, 0.0005, 0.002, 0.0001, 0.005]   # 相對被推薦率比較低的商品  -> 新奇度較高
high_probs = [0.1, 0.05, 0.2, 0.01, 0.5]            # 相對被推薦率比較高的商品  -> 新奇度較低
k = 5


print(f"Novelty@{k}: {novelty_at_k(low_probs, k)}")
print(f"Novelty@{k}: {novelty_at_k(high_probs, k)}")

Novelty@5: 10.165784284662086
Novelty@5: 3.5219280948873624
